## Build Data Mart
#### Contributors:
##### Tean 8: Anthony Ung, Sean Jerzewski, Gideon Kipkorir
##### Team 9: Rohith, Sneha Dasarla
##### Team 10: Anmol Brahmbhatt, Nikita Brahmbhatt, Satya

## 0. Dependencies and Global Variables

In [1]:
import os
from enum import Enum
import csv
import sqlite3 as lite
from decimal import Decimal
from datetime import date, datetime, timedelta

## 1. Gather the file paths
  
  
## IMPORTANT: 
#### Most of these files are untracked on GitHub. it is each team members'   
####   &emsp; &emsp; It is each team members' individual responsibilities  
####   &emsp; &emsp; to build the Database and CSV files for themselves using the other Jupyter notebooks. 

In [2]:
FILE_PATHS = {
    'DATA_MART_PATH' : './../01_Source_DBs/Region_C_Data_Mart.db',
    'PRODUCTS_TABLE' : './ConformedProducts2.txt'
}


In [3]:
ALL_FILES_OK = True

for file_key in FILE_PATHS:
    file_name = FILE_PATHS[file_key]
    file_exists = os.path.isfile(file_name)
    
    if(file_exists):
        print(f'OK - {file_key} - \'{file_name}\'')
    else:
        ALL_FILES_OK = False
        print(f'MISSING - {file_key} - \'{file_name}\'')

if not ALL_FILES_OK:
    raise SystemExit('\n' "ERROR!" '\n' "You are missing files!" '\n' "Read and Follow the Cell instructions provided.")

OK - DATA_MART_PATH - './../01_Source_DBs/Region_C_Data_Mart.db'
OK - PRODUCTS_TABLE - './ConformedProducts2.txt'


## 2. Initialize the Database API

In [4]:
'''
    If I try to make db_options an inner class to db, 
        I get an error saying that the class is undefined.
'''
class db_options(Enum):
        DEFAULT = 0
        RETURN_RESULTS = 1
        PRINT_RESULTS = 2

class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql, values)


    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

## 3. Build Products Table

In [5]:
TABLE_DEFINITIONS = {
    'products': \
            'CREATE TABLE products(' \
                    'ProductKey1 INT,' \
                    'sku TEXT, ' \
                    'product_name TEXT, ' \
                    'product_class_id TEXT, ' \
                    'subcategory TEXT, ' \
                    'category TEXT, ' \
                    'department TEXT, ' \
                    'product_family TEXT, ' \
                    'size TEXT, ' \
                    'brandName TEXT, ' \
                    'supplier TEXT)',
}

In [15]:
def run():
    db_handle = db(FILE_PATHS['DATA_MART_PATH'])

    db_handle.connect()
    db_handle.build_table('products')
    db_handle.commit()

    csv.register_dialect('tab', delimiter='\t', quoting=csv.QUOTE_NONE)

    with open(FILE_PATHS['PRODUCTS_TABLE'], 'r') as csvfile:
        for row in csv.DictReader(csvfile, dialect='tab'):
            ProductKey1 = row.get('ProductKey1')
            sku = row.get('sku')
            product_name = row.get('product_name')
            product_class_id = row.get('product_class_id')
            subcategory = row.get('subcategory')
            category = row.get('category')
            department = row.get('department')
            product_family = row.get('product_family')
            size = row.get('size')
            brandName = row.get('brandName')
            supplier = row.get('supplier')

            db_handle.execute_sql_values( \
                sql='insert into products values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',\
                values=(ProductKey1, sku, product_name, product_class_id, subcategory, category, department, product_family, size, brandName, supplier)
            )
    
    db_handle.commit()
    db_handle.close()


In [16]:
run()